**Problem name: LearnX sales forcasting**

**Problem type: Regression**

In [ ]:
#Import lib
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#reading data
train= pd.read_csv('/kaggle/input/womenintheloop-data-science-hackathon/train.csv')
test= pd.read_csv('/kaggle/input/womenintheloop-data-science-hackathon/test_QkPvNLx.csv')
sample= pd.read_csv('/kaggle/input/womenintheloop-data-science-hackathon/sample_submission_pn2DrMq.csv')

print(train.shape, test.shape, sample.shape)

In [ ]:
# check columns
print(train.columns)
print('-------------')
print(test.columns)

**Apart from target field: Sales, User traffic column is also missing in test dataframe.**

In [ ]:
# take a look at the data
train.head()

1. **Analysis on Dependent Variable: Sales**

In [ ]:
# Descriptive Stats
train.Sales.describe()

In [ ]:
train[['Sales']].boxplot()

In [ ]:
#histogram
sns.distplot(train['Sales'])

* Deviation from normal distibution
* Positive skewness (most values lesser than its average sales value)

In [ ]:
# Positive skewness also mean, mean and median > mode
#mean, median, mode
print(train.Sales.mean())
print(train.Sales.median())
print(train.Sales.mode()[0])

In [ ]:
#skewness and kurtosis
print("Skewness= ", train['Sales'].skew())
print("Kurtosis= ", train['Sales'].kurt())

* Skewness > 1 : shows target is highly skewed
* kurtosis > 3 : Distribution is longer, Many Outliers

[Skewness and Kurtosis Article](https://codeburst.io/2-important-statistics-terms-you-need-to-know-in-data-science-skewness-and-kurtosis-388fef94eeaa)

**2. Sales relation with other cols**

In [ ]:
#datatypes
train.dtypes

In [ ]:
# ID vs Sales
#sns.regplot(x='ID', y='Sales', data=train)

In [ ]:
train.ID.nunique()

In [ ]:
train['ID'].hist()

* ID has unique value for each row and no relation to sales, Need to drop it

In [ ]:
# Day_No
train.Day_No.nunique()

In [ ]:
train['Day_No'].hist()

In [ ]:
sns.regplot(x='Day_No', y='Sales', data= train, ci=None)

In [ ]:
# plotting on sample of dataset coz data is huge 
sampletrain= train.sample(1000)
sns.regplot(x='Day_No',y='Sales',data= sampletrain)

* Not a good estimator

In [ ]:
#3. Course_ID
train.Course_ID.nunique()

In [ ]:
sns.regplot(x='Course_ID', y='Sales', data=sampletrain)

In [ ]:
#4. Short_Promotion
train.Short_Promotion.nunique()

In [ ]:
train.Short_Promotion.hist()

In [ ]:
sns.boxplot(x='Short_Promotion',y='Sales',data=train)

In [ ]:
sns.boxplot(x='Long_Promotion',y='Sales',data=train)

In [ ]:
sns.boxplot(x='Course_Domain',y='Sales',data=train)

In [ ]:
sns.boxplot(x='Course_Type',y='Sales',data=train)

In [ ]:
sns.boxplot(x='Public_Holiday',y='Sales',data=train)

Correlation Matrix

In [ ]:
plt.subplots(figsize=(10,8))
sns.heatmap(train.corr(), annot= True)

Not much correlation exist. Only User traffic has good correlation with sales.

In [ ]:
#pairplot- high comput dont run

#sns.pairplot(train)

**Missing Values**
* How much missing value?
* Missing random or does it has some pattern?

In [ ]:
# train
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

In [ ]:
# test
total_test = test.isnull().sum().sort_values(ascending=False)
percent_test = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data_test = pd.concat([total_test, percent_test], axis=1, keys=['Total', 'Percent'])
missing_data_test

In [ ]:
sns.heatmap(train.isnull())

Data standardization means converting data values to have mean of 0 and a standard deviation of 1

In [ ]:
#standardizing data
from sklearn.preprocessing import StandardScaler
sales_scaled = StandardScaler().fit_transform(train['Sales'][:,np.newaxis])
low_range = sales_scaled[sales_scaled[:,0].argsort()][:10]
high_range= sales_scaled[sales_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

* Low range values are similar and not very far from 0
* Higher range values are very far and out of range.

**Normalisation**

In [ ]:
#histogram and normal probability plot

from scipy.stats import norm
from scipy import stats

sns.distplot(train['Sales'], fit=norm)
fig = plt.figure()
res = stats.probplot(train['Sales'], plot=plt)

* Sales not normal, positive skewness, log transformation works in case of positive skewness.

In [ ]:
# to count the number of 0 in Sales
(train['Sales']==0).sum()

Log transform will result into undefined value in case of 0, Either mask these 0 values before log transform or use (log + 1) transformation.

In [ ]:
#applying log+1 transformation
train['Sales'] = np.log1p(train['Sales'])

In [ ]:
#transformed histogram and normal probability plot
sns.distplot(train['Sales'], fit=norm)
fig = plt.figure()
res = stats.probplot(train['Sales'], plot=plt)